In [1]:
"""
Comprehensive Exploratory Data Analysis for Life Expectancy Dataset
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr, spearmanr, shapiro, normaltest
import os
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# ── Windows output directory ──────────────────────────────────────────────────
OUTPUT_DIR = r"C:\Users\sejal\OneDrive\Desktop\Comprehensive EDA"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def save_fig(filename: str) -> str:
    """Save figure to the Windows output directory and return the full path."""
    path = os.path.join(OUTPUT_DIR, filename)
    plt.savefig(path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"   📊 Visualization saved: {path}")
    return path


class LifeExpectancyEDA:
    """
    Comprehensive EDA class for Life Expectancy dataset
    """

    def __init__(self, filepath):
        """Load and initialize the dataset"""
        print("=" * 100)
        print("LOADING LIFE EXPECTANCY DATASET")
        print("=" * 100)
        self.df = pd.read_csv(filepath)
        self.df_original = self.df.copy()
        print(f"✓ Dataset loaded successfully!")
        print(f"  Shape: {self.df.shape[0]} rows × {self.df.shape[1]} columns\n")

    # ── 1. BASIC INFO ──────────────────────────────────────────────────────────
    def basic_info(self):
        """Display basic dataset information"""
        print("=" * 100)
        print("1. BASIC DATASET INFORMATION")
        print("=" * 100)

        print("\n📊 Dataset Overview:")
        print(f"   • Total Records: {len(self.df):,}")
        print(f"   • Total Features: {len(self.df.columns)}")
        print(f"   • Memory Usage: {self.df.memory_usage(deep=True).sum() / 1024 ** 2:.2f} MB")

        print("\n📋 Column Information:")
        info_df = pd.DataFrame({
            'Column': self.df.columns,
            'Type': self.df.dtypes.values,
            'Non-Null Count': self.df.count().values,
            'Null Count': self.df.isnull().sum().values,
            'Null %': (self.df.isnull().sum() / len(self.df) * 100).round(2).values,
            'Unique Values': [self.df[col].nunique() for col in self.df.columns]
        })
        print(info_df.to_string(index=False))

        print("\n📈 Data Types Distribution:")
        print(self.df.dtypes.value_counts())

        return info_df

    # ── 2. MISSING VALUES ──────────────────────────────────────────────────────
    def missing_value_analysis(self):
        """Comprehensive missing value analysis"""
        print("\n" + "=" * 100)
        print("2. MISSING VALUE ANALYSIS")
        print("=" * 100)

        missing = self.df.isnull().sum()
        missing_pct = (missing / len(self.df) * 100).round(2)
        missing_df = pd.DataFrame({
            'Column': missing.index,
            'Missing_Count': missing.values,
            'Missing_Percentage': missing_pct.values
        }).sort_values('Missing_Count', ascending=False)

        print("\n🔍 Missing Values Summary:")
        print(missing_df[missing_df['Missing_Count'] > 0].to_string(index=False))

        if missing_df['Missing_Count'].sum() == 0:
            print("\n✓ No missing values detected!")
        else:
            print(f"\n⚠ Total missing values: {missing_df['Missing_Count'].sum():,}")
            print(f"⚠ Columns with missing data: {(missing_df['Missing_Count'] > 0).sum()}")

            fig, axes = plt.subplots(1, 2, figsize=(15, 5))

            missing_cols = missing_df[missing_df['Missing_Count'] > 0].head(10)
            if len(missing_cols) > 0:
                axes[0].barh(missing_cols['Column'], missing_cols['Missing_Percentage'])
                axes[0].set_xlabel('Missing Percentage (%)')
                axes[0].set_title('Top 10 Columns with Missing Values')
                axes[0].grid(axis='x', alpha=0.3)

            plt.sca(axes[1])
            sns.heatmap(self.df.isnull(), cbar=False, yticklabels=False, cmap='viridis')
            axes[1].set_title('Missing Value Pattern')

            plt.tight_layout()
            save_fig('01_missing_values.png')

        return missing_df

    # ── 3. STATISTICAL SUMMARY ────────────────────────────────────────────────
    def statistical_summary(self):
        """Comprehensive statistical summary"""
        print("\n" + "=" * 100)
        print("3. STATISTICAL SUMMARY")
        print("=" * 100)

        numerical_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()

        print("\n📊 Descriptive Statistics for Numerical Variables:")
        desc = self.df[numerical_cols].describe()
        print(desc.round(2))

        print("\n📈 Additional Statistical Measures:")
        additional_stats = pd.DataFrame({
            'Column': numerical_cols,
            'Median': [self.df[col].median() for col in numerical_cols],
            'Mode': [self.df[col].mode()[0] if len(self.df[col].mode()) > 0 else np.nan
                     for col in numerical_cols],
            'Skewness': [self.df[col].skew() for col in numerical_cols],
            'Kurtosis': [self.df[col].kurtosis() for col in numerical_cols],
            'CV': [self.df[col].std() / self.df[col].mean() * 100
                   if self.df[col].mean() != 0 else np.nan
                   for col in numerical_cols]
        })
        print(additional_stats.round(3).to_string(index=False))

        return desc, additional_stats

    # ── 4. TARGET VARIABLE ────────────────────────────────────────────────────
    def target_variable_analysis(self, target='LifeExpectancy'):
        """Deep analysis of target variable (Life Expectancy)"""
        print("\n" + "=" * 100)
        print("4. TARGET VARIABLE ANALYSIS (LIFE EXPECTANCY)")
        print("=" * 100)

        if target not in self.df.columns:
            print(f"⚠ Warning: '{target}' column not found!")
            return

        y = self.df[target].dropna()

        print(f"\n📊 Life Expectancy Statistics:")
        print(f"   • Mean: {y.mean():.2f} years")
        print(f"   • Median: {y.median():.2f} years")
        print(f"   • Std Dev: {y.std():.2f} years")
        print(f"   • Min: {y.min():.2f} years")
        print(f"   • Max: {y.max():.2f} years")
        print(f"   • Range: {y.max() - y.min():.2f} years")
        print(f"   • IQR: {y.quantile(0.75) - y.quantile(0.25):.2f} years")
        print(f"   • Skewness: {y.skew():.3f}")
        print(f"   • Kurtosis: {y.kurtosis():.3f}")

        print(f"\n🔬 Normality Tests:")
        shapiro_stat, shapiro_p = shapiro(y.sample(min(5000, len(y))))
        print(f"   • Shapiro-Wilk: statistic={shapiro_stat:.4f}, p-value={shapiro_p:.4f}")

        fig, axes = plt.subplots(2, 3, figsize=(18, 10))

        # Histogram with KDE
        axes[0, 0].hist(y, bins=50, edgecolor='black', alpha=0.7, density=True)
        y.plot(kind='kde', ax=axes[0, 0], color='red', linewidth=2)
        axes[0, 0].set_xlabel('Life Expectancy (years)')
        axes[0, 0].set_ylabel('Density')
        axes[0, 0].set_title(
            f'Distribution of Life Expectancy\nMean: {y.mean():.1f}, Median: {y.median():.1f}')
        axes[0, 0].axvline(y.mean(), color='blue', linestyle='--', label='Mean')
        axes[0, 0].axvline(y.median(), color='green', linestyle='--', label='Median')
        axes[0, 0].legend()

        # Box plot
        axes[0, 1].boxplot(y, vert=True)
        axes[0, 1].set_ylabel('Life Expectancy (years)')
        axes[0, 1].set_title('Box Plot - Outlier Detection')
        axes[0, 1].grid(axis='y', alpha=0.3)

        # Q-Q plot
        stats.probplot(y, dist="norm", plot=axes[0, 2])
        axes[0, 2].set_title('Q-Q Plot - Normality Check')

        # Violin plot
        axes[1, 0].violinplot([y], vert=True, showmeans=True, showmedians=True)
        axes[1, 0].set_ylabel('Life Expectancy (years)')
        axes[1, 0].set_title('Violin Plot')
        axes[1, 0].grid(axis='y', alpha=0.3)

        # Time series
        if 'Year' in self.df.columns:
            yearly_life_exp = self.df.groupby('Year')[target].mean()
            axes[1, 1].plot(yearly_life_exp.index, yearly_life_exp.values,
                            marker='o', linewidth=2)
            axes[1, 1].set_xlabel('Year')
            axes[1, 1].set_ylabel('Average Life Expectancy')
            axes[1, 1].set_title('Life Expectancy Trend Over Years')
            axes[1, 1].grid(alpha=0.3)

        # CDF
        sorted_y = np.sort(y)
        cumulative = np.arange(1, len(sorted_y) + 1) / len(sorted_y)
        axes[1, 2].plot(sorted_y, cumulative, linewidth=2)
        axes[1, 2].set_xlabel('Life Expectancy (years)')
        axes[1, 2].set_ylabel('Cumulative Probability')
        axes[1, 2].set_title('Cumulative Distribution Function')
        axes[1, 2].grid(alpha=0.3)

        plt.tight_layout()
        save_fig('02_target_analysis.png')

    # ── 5. CATEGORICAL ANALYSIS ───────────────────────────────────────────────
    # FIX 1: method body was de-dented (treated as module-level code).
    # FIX 2: Removed the dangling `categorical_analysis_enhanced` stub whose
    #         body was also wrongly de-dented and mixed into the module scope.
    def categorical_analysis(self):
        """Analysis of categorical variables"""
        print("\n" + "=" * 100)
        print("5. CATEGORICAL VARIABLE ANALYSIS")
        print("=" * 100)

        categorical_cols = self.df.select_dtypes(include=['object']).columns.tolist()

        if len(categorical_cols) == 0:
            print("No categorical variables found.")
            return

        print(f"\n📋 Found {len(categorical_cols)} categorical variables: {categorical_cols}")

        for col in categorical_cols:
            print(f"\n{'─' * 80}")
            print(f"Variable: {col}")
            print(f"{'─' * 80}")
            value_counts = self.df[col].value_counts()
            print(f"   • Unique values: {self.df[col].nunique()}")
            print(f"   • Most common: {value_counts.index[0]} ({value_counts.iloc[0]} occurrences)")
            print(f"\n   Distribution:")
            for idx, count in value_counts.items():
                percentage = (count / len(self.df) * 100)
                print(f"      {idx}: {count} ({percentage:.1f}%)")

        n_cats = len(categorical_cols)
        if n_cats > 0:
            n_rows = (n_cats + 1) // 2
            fig, axes = plt.subplots(n_rows, 2, figsize=(16, 5 * n_rows))

            if n_cats == 1:
                axes = np.array([[axes]])
            axes = axes.flatten()

            for idx, col in enumerate(categorical_cols):
                value_counts = self.df[col].value_counts()

                if col == 'Country' and len(value_counts) > 20:
                    value_counts = value_counts.head(20)
                    title = f'{col} Distribution (Top 20)'
                else:
                    title = f'{col} Distribution'

                bars = axes[idx].bar(range(len(value_counts)), value_counts.values,
                                     color='steelblue', edgecolor='black',
                                     linewidth=0.5, alpha=0.8)

                max_idx = value_counts.values.argmax()
                bars[max_idx].set_color('coral')

                axes[idx].set_xticks(range(len(value_counts)))
                max_label_length = max(len(str(label)) for label in value_counts.index)
                if len(value_counts) > 10 or max_label_length > 8:
                    axes[idx].set_xticklabels(value_counts.index, rotation=45,
                                              ha='right', fontsize=9)
                else:
                    axes[idx].set_xticklabels(value_counts.index, rotation=0,
                                              fontsize=10)

                axes[idx].set_ylabel('Count', fontsize=11, fontweight='bold')
                axes[idx].set_title(title, fontsize=12, fontweight='bold', pad=10)
                axes[idx].grid(axis='y', alpha=0.3, linestyle='--')

                if len(value_counts) <= 15:
                    for i, v in enumerate(value_counts.values):
                        axes[idx].text(i, v, str(v), ha='center', va='bottom',
                                       fontsize=9, fontweight='bold')

                if len(value_counts) <= 5:
                    for i, (label, count) in enumerate(value_counts.items()):
                        pct = (count / len(self.df)) * 100
                        axes[idx].text(i, count / 2, f'{pct:.1f}%',
                                       ha='center', va='center',
                                       fontsize=10, fontweight='bold', color='white')

            for idx in range(n_cats, len(axes)):
                axes[idx].axis('off')

            plt.tight_layout(pad=2.0)
            save_fig('03_categorical_analysis.png')

    # ── 6. CORRELATION ANALYSIS ───────────────────────────────────────────────
    def correlation_analysis(self, target='LifeExpectancy'):
        """Comprehensive correlation analysis"""
        print("\n" + "=" * 100)
        print("6. CORRELATION ANALYSIS")
        print("=" * 100)

        numerical_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()

        if len(numerical_cols) < 2:
            print("Not enough numerical variables for correlation analysis.")
            return

        print("\n📊 Pearson Correlation Matrix (Linear Relationships):")
        corr_matrix = self.df[numerical_cols].corr(method='pearson')
        print(corr_matrix.round(3))

        if target in numerical_cols:
            print(f"\n🎯 Correlation with Target Variable ({target}):")
            target_corr = corr_matrix[target].sort_values(ascending=False)
            print(target_corr.round(3))

            print(f"\n🔝 Top 10 Positively Correlated Features:")
            print(target_corr[target_corr > 0].head(11).to_string())

            print(f"\n🔻 Top 10 Negatively Correlated Features:")
            print(target_corr[target_corr < 0].tail(10).to_string())

        fig, axes = plt.subplots(1, 2, figsize=(20, 8))

        sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                    square=True, linewidths=0.5, cbar_kws={"shrink": 0.8}, ax=axes[0])
        axes[0].set_title('Pearson Correlation Heatmap (All Variables)',
                          fontsize=14, fontweight='bold')

        if target in numerical_cols:
            target_corr_sorted = target_corr.drop(target).sort_values()
            colors = ['red' if x < 0 else 'green' for x in target_corr_sorted.values]
            axes[1].barh(range(len(target_corr_sorted)), target_corr_sorted.values,
                         color=colors, alpha=0.7)
            axes[1].set_yticks(range(len(target_corr_sorted)))
            axes[1].set_yticklabels(target_corr_sorted.index, fontsize=8)
            axes[1].set_xlabel('Correlation Coefficient')
            axes[1].set_title(f'Feature Correlation with {target}',
                              fontsize=14, fontweight='bold')
            axes[1].axvline(0, color='black', linewidth=0.8)
            axes[1].grid(axis='x', alpha=0.3)

        plt.tight_layout()
        save_fig('04_correlation_analysis.png')

        print("\n⚠ Multicollinearity Detection (High Correlations between Features):")
        high_corr_pairs = []
        for i in range(len(corr_matrix.columns)):
            for j in range(i + 1, len(corr_matrix.columns)):
                if abs(corr_matrix.iloc[i, j]) > 0.8:
                    high_corr_pairs.append({
                        'Feature 1': corr_matrix.columns[i],
                        'Feature 2': corr_matrix.columns[j],
                        'Correlation': corr_matrix.iloc[i, j]
                    })

        if high_corr_pairs:
            high_corr_df = pd.DataFrame(high_corr_pairs).sort_values(
                'Correlation', ascending=False)
            print(high_corr_df.to_string(index=False))
            print(f"\n⚠ Warning: {len(high_corr_pairs)} feature pairs with |correlation| > 0.8 detected!")
            print("   Consider removing one feature from each pair to avoid multicollinearity.")
        else:
            print("✓ No severe multicollinearity detected (no pairs with |correlation| > 0.8)")

        return corr_matrix, high_corr_pairs

    # ── 7. OUTLIER DETECTION ──────────────────────────────────────────────────
    def outlier_detection(self):
        """Comprehensive outlier detection"""
        print("\n" + "=" * 100)
        print("7. OUTLIER DETECTION")
        print("=" * 100)

        numerical_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()
        outlier_summary = []

        for col in numerical_cols:
            data = self.df[col].dropna()
            Q1 = data.quantile(0.25)
            Q3 = data.quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers_iqr = data[(data < lower_bound) | (data > upper_bound)]
            outlier_pct = (len(outliers_iqr) / len(data)) * 100
            z_scores = np.abs(stats.zscore(data))
            outliers_zscore = data[z_scores > 3]

            outlier_summary.append({
                'Feature': col,
                'IQR_Outliers': len(outliers_iqr),
                'IQR_Outlier_%': outlier_pct,
                'Z-Score_Outliers': len(outliers_zscore),
                'Lower_Bound': lower_bound,
                'Upper_Bound': upper_bound
            })

        outlier_df = pd.DataFrame(outlier_summary).sort_values('IQR_Outliers', ascending=False)
        print("\n📊 Outlier Summary (IQR Method):")
        print(outlier_df.round(2).to_string(index=False))

        n_cols = len(numerical_cols)
        n_rows = (n_cols + 2) // 3
        fig, axes = plt.subplots(n_rows, 3, figsize=(18, 5 * n_rows))
        axes = axes.flatten() if n_cols > 1 else [axes]

        for idx, col in enumerate(numerical_cols):
            axes[idx].boxplot(self.df[col].dropna(), vert=True)
            axes[idx].set_ylabel(col)
            axes[idx].set_title(
                f'{col}\n'
                f'({outlier_df[outlier_df["Feature"] == col]["IQR_Outliers"].values[0]} outliers)'
            )
            axes[idx].grid(axis='y', alpha=0.3)

        for idx in range(n_cols, len(axes)):
            axes[idx].axis('off')

        plt.tight_layout()
        save_fig('05_outlier_detection.png')

        return outlier_df

    # ── 8. FEATURE DISTRIBUTIONS ──────────────────────────────────────────────
    def feature_distributions(self):
        """Analyze distributions of all numerical features"""
        print("\n" + "=" * 100)
        print("8. FEATURE DISTRIBUTION ANALYSIS")
        print("=" * 100)

        numerical_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()
        print(f"\n📊 Analyzing distributions for {len(numerical_cols)} numerical features...")

        n_cols = len(numerical_cols)
        n_rows = (n_cols + 2) // 3
        fig, axes = plt.subplots(n_rows, 3, figsize=(18, 5 * n_rows))
        axes = axes.flatten() if n_cols > 1 else [axes]

        for idx, col in enumerate(numerical_cols):
            data = self.df[col].dropna()
            axes[idx].hist(data, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
            axes[idx].set_xlabel(col)
            axes[idx].set_ylabel('Frequency')
            axes[idx].set_title(f'{col}\nSkew: {data.skew():.2f}, Kurt: {data.kurtosis():.2f}')
            axes[idx].axvline(data.mean(), color='red', linestyle='--',
                              linewidth=2, label='Mean')
            axes[idx].axvline(data.median(), color='green', linestyle='--',
                              linewidth=2, label='Median')
            axes[idx].legend()
            axes[idx].grid(axis='y', alpha=0.3)

        for idx in range(n_cols, len(axes)):
            axes[idx].axis('off')

        plt.tight_layout()
        save_fig('06_feature_distributions.png')

    # ── 9. BIVARIATE ANALYSIS ─────────────────────────────────────────────────
    def bivariate_analysis(self, target='LifeExpectancy'):
        """Bivariate analysis between features and target"""
        print("\n" + "=" * 100)
        print("9. BIVARIATE ANALYSIS (Features vs Target)")
        print("=" * 100)

        if target not in self.df.columns:
            print(f"⚠ Warning: Target variable '{target}' not found!")
            return

        numerical_cols = [col for col in self.df.select_dtypes(include=[np.number]).columns
                          if col != target]

        corr_with_target = self.df[numerical_cols + [target]].corr()[target].drop(target)
        top_features = corr_with_target.abs().sort_values(ascending=False).head(9).index.tolist()

        print(f"\n📊 Analyzing top {len(top_features)} features most correlated with {target}:")
        print(corr_with_target[top_features].to_string())

        n_features = len(top_features)
        n_rows = (n_features + 2) // 3
        fig, axes = plt.subplots(n_rows, 3, figsize=(18, 5 * n_rows))
        axes = axes.flatten() if n_features > 1 else [axes]

        for idx, feature in enumerate(top_features):
            x = self.df[feature].dropna()
            y = self.df.loc[x.index, target]

            axes[idx].scatter(x, y, alpha=0.5, s=20, edgecolors='black', linewidths=0.5)

            z = np.polyfit(x, y, 1)
            p = np.poly1d(z)
            axes[idx].plot(x, p(x), "r--", linewidth=2,
                           label=f'y={z[0]:.2f}x+{z[1]:.2f}')

            axes[idx].set_xlabel(feature)
            axes[idx].set_ylabel(target)
            axes[idx].set_title(
                f'{feature} vs {target}\nr = {corr_with_target[feature]:.3f}')
            axes[idx].legend()
            axes[idx].grid(alpha=0.3)

        for idx in range(n_features, len(axes)):
            axes[idx].axis('off')

        plt.tight_layout()
        save_fig('07_bivariate_analysis.png')

    # ── 10. TEMPORAL ANALYSIS ─────────────────────────────────────────────────
    # FIX 3: `temporal_analysis` was defined at MODULE level (missing `self` and
    #         wrong indentation). Moved inside the class and fixed the loop body
    #         indentation so all axes[idx] calls are inside the for-loop.
    def temporal_analysis(self):
        """Temporal analysis if Year column exists"""
        print("\n" + "=" * 100)
        print("10. TEMPORAL ANALYSIS")
        print("=" * 100)

        if 'Year' not in self.df.columns:
            print("⚠ No 'Year' column found. Skipping temporal analysis.")
            return

        print(f"\n📊 Year Range: {self.df['Year'].min()} - {self.df['Year'].max()}")
        print(f"   Total Years: {self.df['Year'].nunique()}")

        numerical_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()
        numerical_cols = [col for col in numerical_cols if col != 'Year']

        priority_features = ['LifeExpectancy', 'InfantMortality', 'AdultMortality',
                             'GDP', 'Schooling', 'HIV_AIDS']
        key_features = [f for f in priority_features if f in numerical_cols][:6]

        if len(key_features) < 6:
            for col in numerical_cols:
                if col not in key_features:
                    key_features.append(col)
                if len(key_features) >= 6:
                    break

        fig, axes = plt.subplots(2, 3, figsize=(20, 10))
        axes = axes.flatten()

        # FIX 4: loop body was de-dented — restored correct indentation
        for idx, feature in enumerate(key_features[:6]):
            yearly_avg = self.df.groupby('Year')[feature].mean()

            axes[idx].plot(yearly_avg.index, yearly_avg.values,
                           marker='o', linewidth=2.5, markersize=8,
                           color='steelblue', label='Average')

            z = np.polyfit(yearly_avg.index, yearly_avg.values, 1)
            p = np.poly1d(z)
            axes[idx].plot(yearly_avg.index, p(yearly_avg.index),
                           "r--", alpha=0.7, linewidth=2, label='Trend')

            axes[idx].set_xlabel('Year', fontsize=11, fontweight='bold')
            axes[idx].set_ylabel(f'Average {feature}', fontsize=11, fontweight='bold')
            axes[idx].set_title(f'{feature} Trend Over Time',
                                fontsize=12, fontweight='bold', pad=10)
            axes[idx].grid(alpha=0.3, linestyle='--')
            axes[idx].legend(loc='best', fontsize=9)

            axes[idx].text(yearly_avg.index[0], yearly_avg.values[0],
                           f'{yearly_avg.values[0]:.1f}',
                           fontsize=9, ha='right', va='bottom')
            axes[idx].text(yearly_avg.index[-1], yearly_avg.values[-1],
                           f'{yearly_avg.values[-1]:.1f}',
                           fontsize=9, ha='left', va='top')

            axes[idx].tick_params(axis='x', rotation=45)

        plt.tight_layout(pad=2.0)
        save_fig('08_temporal_analysis.png')

    # ── 11. REGRESSION READINESS ──────────────────────────────────────────────
    # FIX 5: `regression_readiness_check` was at module level — moved inside class.
    def regression_readiness_check(self, target='LifeExpectancy'):
        """Check dataset readiness for regression modeling"""
        print("\n" + "=" * 100)
        print("11. REGRESSION MODELING READINESS CHECK")
        print("=" * 100)

        checks = {
            'Target Variable Present': target in self.df.columns,
            'Sufficient Data Points': len(self.df) >= 100,
            'Multiple Features Available': len(
                self.df.select_dtypes(include=[np.number]).columns) > 2,
            'No Excessive Missing Values': (
                self.df.isnull().sum() / len(self.df)).max() < 0.5,
            'Target Variable Continuous': (
                self.df[target].dtype in [np.float64, np.int64]
                if target in self.df.columns else False
            ),
        }

        print("\n✓ Regression Readiness Checklist:")
        for check, status in checks.items():
            symbol = "✓" if status else "✗"
            print(f"   {symbol} {check}: {'PASS' if status else 'FAIL'}")

        all_passed = all(checks.values())

        if all_passed:
            print("\n🎉 Dataset is READY for regression modeling!")
        else:
            print("\n⚠ Dataset requires preprocessing before regression modeling.")

        print("\n📋 Recommendations for Regression Modeling:")
        print("   1. LINEAR REGRESSION:")
        print("      • Use for simple relationships")
        print("      • Select single most correlated feature")
        print("      • Check linearity assumption")

        print("\n   2. MULTIPLE LINEAR REGRESSION:")
        print("      • Use all relevant features")
        print("      • Handle multicollinearity (VIF < 10)")
        print("      • Consider feature scaling")
        print("      • Remove redundant features")

        print("\n   3. POLYNOMIAL REGRESSION:")
        print("      • Use when relationships are non-linear")
        print("      • Start with degree 2 or 3")
        print("      • Watch for overfitting")
        print("      • Use regularization if needed")

        return checks

    # ── 12. SUMMARY REPORT ────────────────────────────────────────────────────
    # FIX 6: `generate_summary_report` was at module level — moved inside class.
    def generate_summary_report(self):
        """Generate comprehensive summary report"""
        print("\n" + "=" * 100)
        print("12. COMPREHENSIVE EDA SUMMARY REPORT")
        print("=" * 100)

        print("\n📊 DATASET OVERVIEW:")
        print(f"   • Total Records: {len(self.df):,}")
        print(f"   • Total Features: {len(self.df.columns)}")
        print(f"   • Numerical Features: {len(self.df.select_dtypes(include=[np.number]).columns)}")
        print(f"   • Categorical Features: {len(self.df.select_dtypes(include=['object']).columns)}")

        print("\n📈 DATA QUALITY:")
        missing_pct = (self.df.isnull().sum().sum() /
                       (len(self.df) * len(self.df.columns))) * 100
        print(f"   • Overall Missing Data: {missing_pct:.2f}%")
        print(f"   • Columns with Missing Data: {(self.df.isnull().sum() > 0).sum()}")

        print("\n🎯 TARGET VARIABLE (LifeExpectancy):")
        if 'LifeExpectancy' in self.df.columns:
            print(f"   • Mean: {self.df['LifeExpectancy'].mean():.2f} years")
            print(
                f"   • Range: {self.df['LifeExpectancy'].min():.2f} - "
                f"{self.df['LifeExpectancy'].max():.2f} years"
            )
            print(f"   • Std Dev: {self.df['LifeExpectancy'].std():.2f} years")

        print("\n🔍 KEY INSIGHTS:")
        print("   • Strong correlations identified between health indicators and life expectancy")
        print("   • Temporal trends show improvement in global health metrics")
        print("   • Dataset suitable for multiple regression approaches")

        print("\n✅ NEXT STEPS:")
        print("   1. Handle missing values (imputation or removal)")
        print("   2. Encode categorical variables")
        print("   3. Scale numerical features")
        print("   4. Split data into train/test sets")
        print("   5. Build and compare regression models:")
        print("      • Simple Linear Regression")
        print("      • Multiple Linear Regression")
        print("      • Polynomial Regression (degree 2-3)")
        print("   6. Evaluate models using R², RMSE, MAE")
        print("   7. Perform residual analysis")
        print("   8. Fine-tune best performing model")

    # ── MAIN RUNNER ───────────────────────────────────────────────────────────
    # FIX 7: `run_complete_eda` was at module level — moved inside class.
    def run_complete_eda(self):
        """Run all EDA analyses"""
        print("\n" + "🔬" * 50)
        print("STARTING COMPREHENSIVE EDA FOR LIFE EXPECTANCY DATASET")
        print("🔬" * 50 + "\n")

        self.basic_info()
        self.missing_value_analysis()
        self.statistical_summary()
        self.target_variable_analysis()
        self.categorical_analysis()
        self.correlation_analysis()
        self.outlier_detection()
        self.feature_distributions()
        self.bivariate_analysis()
        self.temporal_analysis()
        self.regression_readiness_check()
        self.generate_summary_report()

        print("\n" + "=" * 100)
        print("✅ COMPREHENSIVE EDA COMPLETED SUCCESSFULLY!")
        print("=" * 100)
        print(f"\n📊 All visualizations saved in: {OUTPUT_DIR}")
        print("   • 01_missing_values.png")
        print("   • 02_target_analysis.png")
        print("   • 03_categorical_analysis.png")
        print("   • 04_correlation_analysis.png")
        print("   • 05_outlier_detection.png")
        print("   • 06_feature_distributions.png")
        print("   • 07_bivariate_analysis.png")
        print("   • 08_temporal_analysis.png")
        print("\n🎉 Ready to proceed with regression modeling!")



In [2]:
# Main execution
if __name__ == "__main__":
    # Initialize EDA
    eda = LifeExpectancyEDA('C:\\Users\\sejal\\OneDrive\\Desktop\\Comprehensive EDA\\life_expectancy_cleaned.csv')
    
    # Run complete EDA
    eda.run_complete_eda()

LOADING LIFE EXPECTANCY DATASET
✓ Dataset loaded successfully!
  Shape: 2204 rows × 5 columns


🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬
STARTING COMPREHENSIVE EDA FOR LIFE EXPECTANCY DATASET
🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬🔬

1. BASIC DATASET INFORMATION

📊 Dataset Overview:
   • Total Records: 2,204
   • Total Features: 5
   • Memory Usage: 0.33 MB

📋 Column Information:
         Column    Type  Non-Null Count  Null Count  Null %  Unique Values
        Country  object            2204           0     0.0            152
           Year   int64            2204           0     0.0             16
 LifeExpectancy float64            2204           0     0.0            340
InfantMortality float64            2204           0     0.0             69
         Status  object            2204           0     0.0              1

📈 Data Types Distribution:
object     2
float64    2
int64      1
Name: count, dtype: int64

2. MISSING VALUE ANALYSIS

🔍 Missing Values Summary:
